In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [4]:
X = df.drop('Outcome',axis=1).values
y = df['Outcome'].values

In [5]:
X.shape

(768, 8)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [9]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

c:\Users\nasrullah\.conda\envs\tf_env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
history = model.fit(X_train,y_train,epochs=100,batch_size=32,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.4699 - loss: 0.7304 - val_accuracy: 0.5584 - val_loss: 0.6872
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6160 - loss: 0.6585 - val_accuracy: 0.6623 - val_loss: 0.6315
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7158 - loss: 0.5995 - val_accuracy: 0.7013 - val_loss: 0.5944
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7326 - loss: 0.5659 - val_accuracy: 0.7013 - val_loss: 0.5701
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7671 - loss: 0.5265 - val_accuracy: 0.7208 - val_loss: 0.5536
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7490 - loss: 0.5363 - val_accuracy: 0.7403 - val_loss: 0.5406
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7412 - loss: 0.5309 - val_accuracy: 0.7273 - val_loss: 0.5303
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7796 - loss: 0.4932 - val_accuracy: 0.7403 

1. How To Select Appropriate Optimizer
2. Number of Nodes in a layer
3. Number of layers in a model
4. All in all one model

In [11]:
# !pip install keras-tuner --upgrade

In [12]:
import keras_tuner as kt

## 1. How To Select Appropriate Optimizer

In [13]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = hp.Choice('optimizer' , values = ['SGD','RMSprop','Adagrad','Adam'])

    model.compile(optimizer = optimizer,loss = 'binary_crossentropy',metrics = ['accuracy'])

    return model

In [14]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5,
                        project_name = 'Optimizer_Select')

In [15]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 07s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 32s


In [16]:
# Get Best Optimizer
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'Adam'}

In [17]:
# Get Best Model
model = tuner.get_best_models(num_models=1)[0]

c:\Users\nasrullah\.conda\envs\tf_env\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [18]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
history = model.fit(X_train,y_train,epochs=100,initial_epoch=5,batch_size=32,validation_data=(X_test,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.7283 - loss: 0.5246 - val_accuracy: 0.7532 - val_loss: 0.5054
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7531 - loss: 0.4936 - val_accuracy: 0.7662 - val_loss: 0.4974
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7384 - loss: 0.4959 - val_accuracy: 0.7857 - val_loss: 0.4921
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7423 - loss: 0.4939 - val_accuracy: 0.7857 - val_loss: 0.4897
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7967 - loss: 0.4534 - val_accuracy: 0.7857 - val_loss: 0.4874
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7744 - loss: 0.4615 - val_accuracy: 0.7857 - val_loss: 0.4870
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7671 - loss: 0.4720 - val_accuracy: 0.7857 - val_loss: 0.4882
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7885 - loss: 0.4536 - val_accuracy: 0

## 2. How to Select Number of Nodes in a layer

In [20]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('units', min_value=8, max_value=128)
    model.add(Dense(units=units, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [21]:
tuner1 = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5,
                        project_name = 'num_neurons'
                        )

Reloading Tuner from .\num_neurons\tuner0.json


In [25]:
tuner1.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

In [26]:
tuner1.get_best_hyperparameters()[0].values

{'num_layers': 3, 'units': 8}

In [28]:
best_hp = tuner1.get_best_hyperparameters()[0]
model = tuner1.hypermodel.build(best_hp)

c:\Users\nasrullah\.conda\envs\tf_env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
model.fit(X_train,y_train,epochs=100,initial_epoch=5,batch_size=32,validation_data=(X_test,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.3414 - loss: 0.9399 - val_accuracy: 0.3377 - val_loss: 0.8615
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3462 - loss: 0.8593 - val_accuracy: 0.3571 - val_loss: 0.8229
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3944 - loss: 0.8077 - val_accuracy: 0.3896 - val_loss: 0.7893
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4442 - loss: 0.7823 - val_accuracy: 0.4545 - val_loss: 0.7581
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5235 - loss: 0.7334 - val_accuracy: 0.5260 - val_loss: 0.7324
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6067 - loss: 0.6840 - val_accuracy: 0.6169 - val_loss: 0.7094
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6300 - loss: 0.6742 - val_accuracy: 0.6104 - val_loss: 0.6927
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6305 - loss: 0.6619 - val_accuracy:

## 3. How To Select Number of layers in a model

In [30]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(117,activation='relu',input_dim=8))

    for i in range(hp.Int('num_layers', min_value = 1 , max_value = 10)):
        model.add(Dense(117,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [31]:
tuner2 = kt.RandomSearch(build_model,
                         objective = 'val_accuracy',
                         max_trials = 5,
                         project_name = 'num_layers')

Reloading Tuner from .\num_layers\tuner0.json


In [32]:
tuner2.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [33]:
tuner2.get_best_hyperparameters()[0].values

{'num_layers': 10}

In [34]:
model = tuner2.get_best_models(num_models=1)[0]

c:\Users\nasrullah\.conda\envs\tf_env\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(X_train,y_train,epochs=100,initial_epoch=5,batch_size=32,validation_data=(X_test,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 51ms/step - accuracy: 0.7786 - loss: 0.4748 - val_accuracy: 0.7338 - val_loss: 0.6187
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7864 - loss: 0.4398 - val_accuracy: 0.7403 - val_loss: 0.7212
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7933 - loss: 0.4643 - val_accuracy: 0.7597 - val_loss: 0.5072
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8242 - loss: 0.4177 - val_accuracy: 0.7338 - val_loss: 0.5374
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7998 - loss: 0.4148 - val_accuracy: 0.7403 - val_loss: 0.5474
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8286 - loss: 0.4025 - val_accuracy: 0.7532 - val_loss: 0.5790
Epoch 12/100


## 4. All in One model

In [ ]:
def build_model(hp):
    model = Sequential()
    counter = 0

    for i in range(hp.Int('num_layers',min_value=1, max_value=10)):
        if counter == 0:
            model.add(
                Dense(
                    hp.Int('units' + str(i),min_value=8 , max_value=128 , step=8),
                    activation = hp.Choice('activation' + str(i) , values = ['relu','tanh','sigmoid']),
                    input_dim = 8
                )
            )
        else:
            model.add(
                Dense(
                    hp.Int('units' + str(i) , min_value = 8 , max_value = 128 , step=8),
                    activation = hp.Choice('activation' + str(i) , values = ['relu','tanh','sigmoid'])
                )
            )
        counter +=1

    model.compile(optimizer=hp.Choice('optimizers',values = ['Adam','RMSprop','SGD','Nadam','Adadelta']),
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy']
                  )

    return model


In [ ]:
tuner3 = kt.RandomSearch(build_model,
                         objective = 'val_accuracy',
                         project_name = 'All_in_One')

c:\Users\nasrullah\.conda\envs\tf_env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner3.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 2 Complete [00h 00m 02s]

Best val_accuracy So Far: None
Total elapsed time: 00h 00m 06s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
1                 |7                 |num_layers
24                |88                |units0
relu              |sigmoid           |activation0
RMSprop           |Adam              |optimizers
56                |8                 |units1
tanh              |relu              |activation1
16                |8                 |units2
relu              |relu              |activation2
48                |8                 |units3
tanh              |relu              |activation3
24                |8                 |units4
sigmoid           |relu              |activation4
80                |8                 |units5
relu              |relu              |activation5
104               |8                 |units6
tanh              |relu              |activation6

Epoch 1/5


Traceback (most recent call last):
  File "c:\Users\nasrullah\.conda\envs\tf_env\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\nasrullah\.conda\envs\tf_env\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nasrullah\.conda\envs\tf_env\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nasrullah\.conda\envs\tf_env\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\nasrullah\.conda\envs\tf_env\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\nasrullah\.conda\envs\tf_env\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nasrullah\.conda\envs\tf_env\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nasrullah\.conda\envs\tf_env\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.f